# CityBikes

In [1]:
import os
import requests
import json
import pandas as pd
import sqlite3

Send a request to CityBikes for the city of your choice. 

- Used the given API endpoint "http://api.citybik.es/v2/networks" and looked through the immesne data available. 
- Stumbled upon my location "Vancouver",my home city, as there was no specific way of filtering the city using the endpoint.
- City Bikes only provided the way of field filteration. Used that to get infomation about bike sharing in Vancouver.

In [ ]:
city_field = "mobibikes"
url = "https://api.citybik.es/v2/networks/"

city_API = requests.get(url + city_field) 

city_JSON = city_API.json()    
print(json.dumps(city_JSON, indent=3))

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [4]:
city = city_JSON['network']['location'] 
print(city)

{'city': 'Vancouver', 'country': 'CA', 'latitude': 49.2827, 'longitude': -123.1207}


In [ ]:
city_stations = city_JSON['network']['stations']
print(city_stations)

Put your parsed results into a DataFrame.

In [6]:
citybikes_df = pd.DataFrame(city_stations)
citybikes_df

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp
0,15,"{'ebikes': 4, 'has_ebikes': True, 'last_update...",21,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2024-03-11T17:04:43.889000Z
1,10,"{'ebikes': 1, 'has_ebikes': True, 'last_update...",6,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2024-03-11T17:04:43.856000Z
2,1,"{'ebikes': 7, 'has_ebikes': True, 'last_update...",25,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2024-03-11T17:04:43.857000Z
3,4,"{'ebikes': 0, 'has_ebikes': True, 'last_update...",12,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2024-03-11T17:04:43.880000Z
4,7,"{'ebikes': 2, 'has_ebikes': True, 'last_update...",9,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2024-03-11T17:04:43.934000Z
...,...,...,...,...,...,...,...,...
246,15,"{'ebikes': 3, 'has_ebikes': True, 'last_update...",5,3dae5d936bb6b222d135eff52833e7f4,49.282047,-123.133057,Bute & Davie,2024-03-11T17:04:43.772000Z
247,6,"{'ebikes': 3, 'has_ebikes': True, 'last_update...",14,10acadd77bc21f26a7f09d8c74d4af2d,49.277773,-123.114237,Beatty & Robson,2024-03-11T17:04:44.065000Z
248,17,"{'ebikes': 2, 'has_ebikes': True, 'last_update...",3,58345bb719f4e570ed4edc9dfa6a68a2,49.263073,-123.254307,West Mall Swing Space Building,2024-03-11T17:04:43.852000Z
249,23,"{'ebikes': 1, 'has_ebikes': True, 'last_update...",3,261efc67071963969b3f848c66454d97,49.259999,-123.250902,Orchard Commons,2024-03-11T17:04:43.675000Z


In [7]:
citybikes_df = pd.json_normalize(city_stations)
citybikes_df

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.renting,extra.returning,extra.slots,extra.uid
0,15,21,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2024-03-11T17:04:43.889000Z,4,True,1710176483,17,True,True,36,0001
1,10,6,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2024-03-11T17:04:43.856000Z,1,True,1710176437,5,True,True,16,0004
2,1,25,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2024-03-11T17:04:43.857000Z,7,True,1710176535,18,True,True,26,0005
3,4,12,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2024-03-11T17:04:43.880000Z,0,True,1710176470,12,True,True,16,0007
4,7,9,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2024-03-11T17:04:43.934000Z,2,True,1710176420,7,True,True,16,0008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,15,5,3dae5d936bb6b222d135eff52833e7f4,49.282047,-123.133057,Bute & Davie,2024-03-11T17:04:43.772000Z,3,True,1710176430,2,True,True,20,0187
247,6,14,10acadd77bc21f26a7f09d8c74d4af2d,49.277773,-123.114237,Beatty & Robson,2024-03-11T17:04:44.065000Z,3,True,1710176274,11,True,True,20,0026
248,17,3,58345bb719f4e570ed4edc9dfa6a68a2,49.263073,-123.254307,West Mall Swing Space Building,2024-03-11T17:04:43.852000Z,2,True,1710176591,1,True,True,20,2114
249,23,3,261efc67071963969b3f848c66454d97,49.259999,-123.250902,Orchard Commons,2024-03-11T17:04:43.675000Z,1,True,1710176328,2,True,True,26,2116


In [8]:
citybikes_df = citybikes_df.drop(['id', 'extra.uid', 'extra.last_updated'], axis=1)

citybikes_df = citybikes_df.drop_duplicates(subset=['latitude', 'longitude'])

citybikes_df = citybikes_df.rename(columns={'extra.ebikes':'ebikes',
                                            'extra.has_bikes':'noOf_ebikes',
                                            'extra.normal_bikes':'noOf_normal_bikes',
                                            'etra.renting':'renting',
                                            'extra.returning':'returning',
                                            'extra.slots':'total_slots'})

# assigning a key value to the table
citybikes_df['key'] = 0
counter = 0
for i, item in citybikes_df.iterrows():
    citybikes_df.loc[i, 'key'] = counter
    counter+=1   
citybikes_df

,empty_slots,free_bikes,latitude,longitude,name,timestamp,ebikes,extra.has_ebikes,noOf_normal_bikes,extra.renting,returning,total_slots,key
0,15,21,49.262487,-123.114397,10th & Cambie,2024-03-11T17:04:43.889000Z,4,True,17,True,True,36,0
1,10,6,49.274566,-123.121817,Yaletown-Roundhouse Station,2024-03-11T17:04:43.856000Z,1,True,5,True,True,16,1
2,1,25,49.279764,-123.110154,Dunsmuir & Beatty,2024-03-11T17:04:43.857000Z,7,True,18,True,True,26,2
3,4,12,49.260599,-123.113504,12th & Yukon (City Hall),2024-03-11T17:04:43.880000Z,0,True,12,True,True,16,3
4,7,9,49.264215,-123.117772,8th & Ash,2024-03-11T17:04:43.934000Z,2,True,7,True,True,16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,15,5,49.282047,-123.133057,Bute & Davie,2024-03-11T17:04:43.772000Z,3,True,2,True,True,20,246
247,6,14,49.277773,-123.114237,Beatty & Robson,2024-03-11T17:04:44.065000Z,3,True,11,True,True,20,247
248,17,3,49.263073,-123.254307,West Mall Swing Space Building,2024-03-11T17:04:43.852000Z,2,True,1,True,True,20,248
249,23,3,49.259999,-123.250902,Orchard Commons,2024-03-11T17:04:43.675000Z,1,True,2,True,True,26,249


In [9]:
# turning the citybikes_df to a csv do it can be used in other notebooks.
citybikes_df.to_csv('citybikes.csv', index=False)